<h1><center>Darwin Supervised Regression Model Building </center></h1>

First, 
<br>if you have just received a new api key from support, you will need to register your key and create a new user (see Register user cell)

Second, in the Environment Variables cell: 
1. Set your username and password to ensure that you're able to log in successfully
2. Set the path to the location of your datasets if you are using your own data.  The path is set for the examples.
3. Set the dataset names accordingly

Here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

### Set Darwin SDK

In [1]:
from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/');

### Environment Variables

In [2]:
#Set your user id and password accordingly
USER="justin.kang23@gmail.com"
PW="psjf52vRYb"

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET='data/'
DATASET_NAME='data.csv'

# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

### Import necessary libraries

In [3]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import r2_score

### User Login 

In [4]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)

### Data Upload

**Read dataset and view a file snippet**

**Upload dataset to Darwin**

In [5]:
def calculate_adults(row):
    return 100-row['% Age 65 and Older']-row['% Age 17 and Under']

In [6]:
def drop_features(df):
    with open('data/remove_col_names.txt', 'r') as f:
        drop_cols=f.read().splitlines()
    return df.drop(columns=drop_cols)

In [70]:
# Preview dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, 'exemptions.csv'))
df.drop(df.tail(1).index,inplace=True)

#combine all data into one dataframe
for file in os.listdir(PATH_TO_DATASET):
    if 'csv' in file and 'exemptions' not in file: #dont add exemptions twice
        df2=pd.read_csv(os.path.join(PATH_TO_DATASET, file))
        df=df.merge(df2, on='County')

#~feature engineering~
df['% Age 18-65']=df.apply (lambda row: calculate_adults(row), axis=1)

df=drop_features(df)

df.to_csv('pruned_data.csv', index=False)

In [61]:
# Upload dataset
status, dataset = ds.upload_dataset('pruned_data.csv')
if not status:
    print(dataset)

### Analyze Data
**Before creating a model, users need to analyze data and clean data first.**

In [ ]:
status, analyze_id = ds.analyze_data('pruned_data.csv', 
                                     job_name = 'Darwin_analyze_data_job' + "-" + ts, 
                                     artifact_name = 'Darwin_analyze_data_artifact' + "-" + ts)
sleep(1)
if status:
    ds.wait_for_job('Darwin_analyze_data_job' + "-" + ts)
else:
    print(analyze_id)

In [ ]:
ds.lookup_job_status_name(analyze_id['job_name'])

**Clean dataset**

In [71]:
# clean dataset
status, job_id = ds.clean_data('pruned_data.csv', impute='ffill')

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'dataset_name': 'pruned.csv'}
{'status': 'Requested', 'starttime': '2019-04-13T19:04:46.412665', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['pruned_data.csv'], 'artifact_names': ['0c10cffd6e9a408cb54adde9b9dea1ba'], 'model_name': None, 'job_error': None}
{'status': 'Complete', 'starttime': '2019-04-13T19:04:46.412665', 'endtime': '2019-04-13T19:04:48.205474', 'percent_complete': 100, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['pruned_data.csv'], 'artifact_names': ['0c10cffd6e9a408cb54adde9b9dea1ba'], 'model_name': None, 'job_error': ''}


In [75]:
ds.download_dataset('pruned_data.csv')

(True,
 {'filename': '/var/folders/kx/xsyth1zs74b9k_5pl3snlxc00000gn/T/pruned_data.csv-part0-_18n96zn.csv',
  'note': 'part 0 of 0',
  'part': 0})

### Create and Train Model 

We will now build a model that will learn the class labels in the target column.<br> In the default boston dataset, the target column is "Assessed_Value". <br> You will have to specify your own target name for your custom dataset. <br> You can also increase max_train_time for longer training.


In [10]:
model = target + "_model0" + ts
status, job_id = ds.create_model(dataset_names = 'pruned_data.csv', \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

400: BAD REQUEST - {"message": "Dataset has not been cleaned. Please clean dataset before training model."}



### Extra Training (Optional)
Run the following cell for extra training, no need to specify parameters

In [11]:
# Train some more
status, job_id = ds.resume_training_model(dataset_names = DATASET_NAME,
                                          model_name = model,
                                          max_train_time = '00:05')
                                          
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

404: NOT FOUND - {"message": "Model name not found"}



### Analyze Model
Analyze model provides feature importance ranked by the model. <br> It indicates a general view of which features pose a bigger impact on the model

In [12]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

404: NOT FOUND - {"message": "Model name not found"}



TypeError: string indices must be integers

Show the 10 most important features of the model.

In [ ]:
feature_importance[:10]

### Predictions
**Perform model prediction on the the training dataset.**

In [ ]:
status, artifact = ds.run_model(DATASET_NAME, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

Download predictions from Darwin's server.

In [ ]:
status, prediction = ds.download_artifact(artifact['artifact_name'])
prediction.head()

Create plots comparing predictions with actual target

In [ ]:
#Plot predictions vs actual
plt.plot(df[target], prediction[target], '.')
plt.plot([0,2.3e7],[0,2.3e7],'--k')
print('R^2 : ', r2_score(df[target], prediction[target]))

### Find out which machine learning model did Darwin use:

In [ ]:
status, model_type = ds.lookup_model_name(model)
print(model_type['description']['best_genome'])